In [1]:
import altair as alt
import jupyterlab_omnisci.vega_ibis
import ibis
import pandas as pd

source = pd.DataFrame({
    'a': ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I'],
    'b': [28, 55, 43, 91, 81, 53, 19, 87, 52]
})
connection = ibis.pandas.connect({'source': source })
table = connection.table('source')

c = alt.Chart(table).mark_bar().encode(
    x='a',
    y='b'
)

In [2]:
c